## Import

In [1]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import albumentations as albu
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models
from torchvision import transforms

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameters Setting

In [3]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':15,
    'LEARNING_RATE':5e-3,
    'BATCH_SIZE':8,
    'SEED':42
}

## Fix RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [5]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            # get image path
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.png')))
            
            # get label
            label_df = pd.read_csv(current_path+'/label.csv')
            label_list.extend(label_df['leaf_weight'])
                
    return np.array(img_path_list), np.array(label_list)

def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0]))
    return np.array(img_path_list)

In [6]:
all_img_path, all_label = get_train_data('../data/train')
test_img_path = get_test_data('../data/test')

## CustomDataset

In [7]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        img = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image=img)["image"]

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [8]:
train_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
#                           rotate_limit=30, interpolation=1, border_mode=0,
#                           value=0, p=0.5),
#     albu.HorizontalFlip(p=0.5),
#     albu.VerticalFlip(p=0.5),
#     albu.RandomRotate90(p=1.0),
#     albu.CLAHE(clip_limit=2, p=0.25),
#     albu.Sharpen(p=0.25),
#     albu.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1),
#                                   contrast_limit=(-0.1, 0.1), p=0.25),
#     albu.RandomResizedCrop(height=CFG['IMG_SIZE'], width=CFG['IMG_SIZE'],
#                            scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333),
#                            interpolation=1, p=1.0),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=255.0, p=1.0),
    ToTensorV2()]
)

valid_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.HorizontalFlip(p=0.5),
#     albu.VerticalFlip(p=0.5),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=255.0, p=1.0),
    ToTensorV2()]
)

test_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=255.0, p=1.0),
    ToTensorV2()]
)

In [9]:
# train_transform = transforms.Compose([
#                     transforms.ToTensor(),
#                     transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
#                     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
#                     ])

# test_transform = transforms.Compose([
#                     transforms.ToTensor(),
#                     transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
#                     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
#                     ])

## Define Model Architecture

In [10]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=1)
        
    def forward(self, x):
        x = self.model(x)
        return x    

In [11]:
class CNNRegressor(torch.nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.regressor = nn.Linear(int((CFG['IMG_SIZE']/16-1)**2*64), 1)


    def forward(self, x):
        # Simple CNN Model (Batch, 3, 128, 128 -> Batch, 64, 7, 7)
        # (Batch, 3, 128, 128)
        x = self.layer1(x)
        # (Batch, 8, 64, 64)
        x = self.layer2(x)
        # (Batch, 16, 32, 32)
        x = self.layer3(x)
        # (Batch, 32, 16, 16)
        x = self.layer4(x)
        # (Batch, 64, 7, 7) -> Flatten (Batch, 64*7*7(=3136))
        x = torch.flatten(x, start_dim=1)
        # Regressor (Batch, 3136) -> (Batch, 1)
        out = self.regressor(x)
        return out

## Train

In [12]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)

    # Loss Function
    criterion = nn.L1Loss().to(device)
    best_mae = 9999
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            # Calc loss
            loss = criterion(logit.squeeze(1), label)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        
        # vali_mae가 더 이상 커지지 않으면
        if scheduler is not None:
            scheduler.step(vali_mae)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), '../model/best_model.pth')
            print('Model Saved.')

In [13]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

## Test Set

In [14]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            
            img = img.float().to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.squeeze(1).detach().cpu()

            model_pred.extend(pred_logit.tolist())
    return model_pred

## Run!!

In [15]:
rows_train = len(all_img_path) # 주어진 train data의 row 수
rows_test = len(test_img_path) # 주어진 test data의 row 수
num_trial = 100 # 파라미터 튜닝을 몇 번 진행하는지의 수
splits_hp = 5 # 파라미터 튜닝을 진행할 때의 kfold 수
splits_tr = 15 # 모델 트레이닝을 진행할 때의 kfold 수
basic_seed = 42 # default seed
num_seed_tr = 10 # 트레이닝 seed 개수
sel_seed = 3 # 선택할 seed 개수

pred_dict = {}
pred_test_dict = {}

In [16]:
# model = CNNRegressor().to(device)
model = Network().to(device)

optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
#                                         lr_lambda=lambda epoch: 0.95 ** epoch)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=5)

In [ ]:
kfold = KFold(n_splits=splits_tr, random_state=basic_seed, shuffle=True) # CV 늘려가면서 하기
cv = np.zeros(rows_train)
pred_test = np.zeros(rows_test)
for n, (train_idx, val_idx) in enumerate(kfold.split(all_img_path, all_label)):
    
    train_img_path, vali_img_path = all_img_path[train_idx], all_img_path[val_idx]
    train_label, vali_label = all_label[train_idx], all_label[val_idx]
    
    # Get Dataloader
    train_dataset = CustomDataset(train_img_path.tolist(), train_label.tolist(), train_mode=True, transforms=train_transform)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_label.tolist(), train_mode=True, transforms=valid_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    train(model, optimizer, train_loader, vali_loader, scheduler, device)
    
    # Validation Score가 가장 뛰어난 모델을 불러옵니다.
    checkpoint = torch.load('../model/best_model.pth')
    model = Network().to(device)
    model.load_state_dict(checkpoint)
    
    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_label.tolist(), train_mode=False, transforms=test_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

    
    cv[val_idx] = predict(model, vali_loader, device)
    pred_test += np.array(predict(model, test_loader, device)) / splits_tr
    print(f"Fold {n} NMAE: {mean_absolute_error(cv[val_idx], vali_label) / np.mean(np.abs(vali_label))}")
    
pred_dict['eff_b0'+str(seed)] = cv
pred_test_dict['eff_b0'+str(seed)] = pred_test

## Submission

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv')
submission['leaf_weight'] = preds
submission_time = datetime.today().strftime('%Y-%m-%d %H:%M')
submission.to_csv(f'../submission/{submission_time}.csv', index = False)

In [ ]:
submission